In [5]:
from DataReader import DataReaderChallangePolimi2017
import implicit
from CB_asy import CB_asymmetric_cosine
from CF_IB_asy import CF_IB_asymmetric_cosine
from CF_UB_asy import CF_UB_asymmetric_cosine
from implicit.nearest_neighbours import (BM25Recommender, CosineRecommender,TFIDFRecommender,bm25_weight)
from sklearn.preprocessing import normalize
import numpy as np
import pickle

evaluation = True
rebuild = False
dr = DataReaderChallangePolimi2017(evaluation=evaluation,rebuild=rebuild)

urm = dr.getURM_csr()
t_p = dr.target_playlists
t_t = dr.target_tracks

Date Reader Challange Polimi 2017
Loading data from ./DumpData/DataReaderTrainTestDump
Dataset train-test loaded


In [14]:
#cf user based
model = BM25Recommender(K=60)
model.fit(urm)
s = model.similarity
s = normalize(s,axis=0)
r_cfub = s.T * urm
#r_cfub = normalize(r_cfub, axis=1)
if evaluation: print dr.evaluateMAP(r_cfub, verbose = False)
r_cfub = dr.reduceRM(r_cfub)

0.0640611111111


In [9]:
#cf item based
model = BM25Recommender(K=100)
model.fit(urm.T)
s = model.similarity
s.setdiag(0)
r_cfib = urm * s.T
r_cfib = normalize(r_cfib, axis=0)
if evaluation: print dr.evaluateMAP(r_cfib, verbose = False)
r_cfib = dr.reduceRM(r_cfib)

0.0746020555556


In [ ]:
0.074602

In [15]:
#total cf
r_tot = 1.85*r_cfib + r_cfub
dr.evaluateMAP(r_tot, verbose = False)

0.078915055555556293

In [ ]:
####### tune k 

In [40]:
#cf ub
def cf_ub(urm,k):
    model = BM25Recommender(K=k)
    model.fit(urm)
    s = model.similarity
    r_cfub = s * urm
    r_cfub = normalize(r_cfub, axis=1) 
    return dr.evaluateMAP(r_cfub, verbose = False)    

In [94]:
#cf ib
def cf_ib(urm,k):
    model = BM25Recommender(K=k)
    model.fit(urm.T)
    s = model.similarity
    r_cfib = urm * s.T
    r_cfib = normalize(r_cfib, axis=0) 
    return dr.evaluateMAP(r_cfib, verbose = False)

In [ ]:
#tune cf ub
urm = dr.getURM_csr()
for i in range(25,95,5):
    score= cf_ub(urm=urm,k=i)
    print ("k = {:3d} \t\tmap = {:1.4f}").format(i,score)

In [ ]:
#tune cf ib
urm = dr.getURM_csr()
for i in range(70,145,5):
    score= cf_ib(urm=urm,k=i)
    print ("k = {:3d} \t\tmap = {:1.4f}").format(i,score)

In [12]:
#tune total
for p in np.arange(1.80, 2.2, 0.05):
    r_tot = p*r_cfib + r_cfub
    score = dr.evaluateMAP(r_tot, verbose = False)
    print ("k = {:1.2f} \t\tmap = {:1.5f}").format(p,score)

k = 1.80 		map = 0.07880
k = 1.85 		map = 0.07892
k = 1.90 		map = 0.07891
k = 1.95 		map = 0.07882
k = 2.00 		map = 0.07882
k = 2.05 		map = 0.07878
k = 2.10 		map = 0.07875
k = 2.15 		map = 0.07881
k = 2.20 		map = 0.07885
